In [164]:
import os

import re
import time
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver import ActionChains

load_dotenv()


# ElementClickInterceptedException - неможна клацнуть по елементу
# NoSuchElementException - такого елементу неіснує

True

In [ ]:
# funcs

def login(driver):
    email = os.getenv("EMAIL")
    password = os.getenv("PASS")
    
    
    login_button = find_elems(driver, {
                                        "by_attr": By.ID, 
                                        "attr_data": "user-menu",
                                        "attr_target": "div"
                                        }
                                )[0]
    login_button.click()
    
    
    block_login_by = find_elems(driver, {
                                        "by_attr": By.CLASS_NAME, 
                                        "attr_data": "lsidDialog__section",
                                        }
                               )
    login_by_email = find_elems(block_login_by[0], {
                                        "by_attr": By.TAG_NAME, 
                                        "attr_data": "button",
                                        }
                               )[-1]
    login_by_email.click()
    
    block_inputs = find_elems(driver, {
                                        "by_attr": By.CLASS_NAME, 
                                        "attr_data": "lsidDialog__section",
                                        }
                               )
    input_fields = find_elems(block_inputs[0], {
                                        "by_attr": By.TAG_NAME, 
                                        "attr_data": "input",
                                        }
                             )
    time.sleep(1)
    input_fields[0].send_keys(email)
    input_fields[1].send_keys(password)
    input_fields[1].send_keys(Keys.RETURN)  
      
    time.sleep(5)  # для закриття вспливаючого вікна
    return driver


def get_driver():
    path_to_driver = "./chromedriver"
    chrome_service = Service(path_to_driver)
    options = Options()
#     options.add_argument("headless")
    return webdriver.Chrome(
        service=chrome_service,
        options=options
    )


def close_cookies(driver):
    """
    закриття вікна з підтвердження куків
    """
    
    elems = find_elems(driver, {
                                "by_attr": By.ID, 
                                "attr_data": "onetrust-accept-btn-handler",
                                }
                      )
    if not elems:
        return driver
    elems[0].click()
    return driver
    
    
def find_elems(target, args):
    args_attr_data = args["attr_data"]
    if args["by_attr"] == By.XPATH and args["by_text"] is False:
        attr_data =  f"//{'attr_target'}[starts-with(@class, '{args_attr_data}')]" 
    elif args["by_attr"] == By.XPATH and args["by_text"] is True:
        attr_data =  f"//{'attr_target'}[text()='{args_attr_data}']"
    else:
        attr_data = args["attr_data"]
    
    count = 0
    while count <= 5:
        try:
            elements = target.find_elements(args["by_attr"], attr_data)
            if len(elements) == 0:
                count += 1
                continue
            break
        except NoSuchElementException:
            time.sleep(1)
            count += 1
    
    return elements

In [166]:

try:
    url = "https://www.soccerstand.com/ru/"
    driver = get_driver()
    driver.get(url)
    driver = close_cookies(driver)
    
    driver = login(driver)
    
    
    time.sleep(5)
    raise
finally:
    driver.quit()


m0rqpEy@proton.me eYgFQnwv46HRTx


RuntimeError: No active exception to reraise